In [ ]:
""" PRELIEVO FILE UTILS UFFICIALI """
# 1. Clona il repository nella cartella di lavoro
!git clone https://github.com/DIUx-xView/data_utilities.git
!pip install protobuf==3.20.*
# 2. (Importante) Aggiungi la cartella al path di Python
# Senza questo passaggio, Python non troverà i file .py appena scaricati
import sys
import os

# Il percorso sarà /kaggle/working/ + nome_repo
repo_path = '/kaggle/working/data_utilities'
sys.path.append(repo_path)

# 3. Verifica
print("File scaricati:", os.listdir(repo_path))

# Ora puoi importare le funzioni come se fossero librerie installate
# import process_data  <-- Esempio di file presente in quel repo

""" IMPORT DEI FILE PRELEVATI """
# 1. Definisci dove sono finiti i file .py (se hai clonato la repo xView)
# Verifica il nome esatto della cartella che si è creata con il comando !ls
cartella_repo = '/kaggle/working/data_utilities' 

# 2. Aggiungi questa cartella al "path" (la lista di dove Python cerca le librerie)
if cartella_repo not in sys.path:
    sys.path.append(cartella_repo)

# 3. ORA puoi importare i file che sono lì dentro
# Esempio: nel repo xView c'è spesso un file chiamato 'geojson_utils.py'
import aug_util as aug
import process_wv
import tfr_util
import wv_util as wv

# Test: Proviamo a vedere se funziona
print("Import riuscito!")

""" ALTRE LIBRERIE """
import matplotlib.pyplot as plt
import numpy as np
import csv
%matplotlib inline

In [ ]:
""" FASE 1: Import e Mappatura Classi
Qui carichiamo le librerie necessarie, definiamo i percorsi e creiamo la mappa per convertire gli ID "strani" di xView (11, 72...) in ID ordinati per YOLO (0, 1, 2...). """
import os
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import wv_util as wv  # La libreria della repo che hai scaricato

# --- CONFIGURAZIONE ---
GEOJSON_PATH = '/kaggle/input/xview-dataset/train_labels/xView_train.geojson'  # Percorso al tuo geojson
IMAGES_DIR = '/kaggle/input/xview-dataset/train_images/train_images'                        # Cartella con le immagini .tif
OUTPUT_DIR = 'datasets/xview_yolo'                 # Dove salvare il dataset pronto
CHIP_SIZE = 640                                    # Dimensione chip (richiesta da te)

def get_id_yolo_mapping(label_file='xview_class_labels.txt'):
    """Crea un dizionario per convertire ID xView -> ID YOLO (0-59)"""
    labels = {}
    with open(label_file) as f:
        for line in f:
            key, val = line.strip().split(':')
            labels[int(key)] = val
            
    # Ordiniamo gli ID originali per garantire coerenza
    sorted_ids = sorted(labels.keys())
    
    # Creiamo la mappa: {11: 0, 12: 1, ...} e la lista nomi
    id_map = {original: new for new, original in enumerate(sorted_ids)}
    names = [labels[original] for original in sorted_ids]
    
    return id_map, names

# Eseguiamo subito la mappatura
ID_MAP, CLASS_NAMES = get_id_yolo_mapping('/kaggle/working/data_utilities/xview_class_labels.txt')
print(f"Mappatura completata. Classi totali: {len(CLASS_NAMES)}")

In [ ]:
""" FASE 2: Creazione Cartelle e Split Train/Val
Qui prepariamo la struttura delle directory e dividiamo i nomi dei file .tif per evitare il Data Leakage.
-----------------NUOVA AGGIUNTA: Test Set per stima in inferenza------------------------------------
"""

def setup_directories():
    """Crea la struttura di cartelle richiesta da YOLO"""
    for split in ['train', 'val', 'test']:
        os.makedirs(f"{OUTPUT_DIR}/{split}/images", exist_ok=True)
        os.makedirs(f"{OUTPUT_DIR}/{split}/labels", exist_ok=True)
    print(f"Cartelle create in {OUTPUT_DIR} (train, val, test)")

def split_dataset(val_percent=0.20, test_percent=0.10):
    """Divide i file .tif in tre liste: train, validation e test"""
    
    # Prendi tutti i file .tif nella cartella immagini
    all_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith('.tif')]

    # Mischia l'ordine (random seed fisso per riproducibilità)
    random.seed(42)
    random.shuffle(all_files)

    total_files = len(all_files)

    # Calcola il numero di file per ogni set
    n_test = int(total_files * test_percent)
    n_val = int(total_files * val_percent)
    # Il train prende tutto il resto (per evitare errori di arrotondamento che lasciano fuori 1 file)
    n_train = total_files - n_val - n_test

    # Slicing delle liste
    # 1. I primi n_train file vanno al Training
    train_files = all_files[:n_train]
    
    # 2. I successivi n_val file vanno alla Validation
    val_files = all_files[n_train : n_train + n_val]
    
    # 3. Gli ultimi n_test file vanno al Test
    test_files = all_files[n_train + n_val :]

    print(f"Totale immagini originali: {total_files}")
    print(f"Training (70%):   {len(train_files)} files")
    print(f"Validation (20%): {len(val_files)} files")
    print(f"Test (10%):       {len(test_files)} files (Da NON patchare, per SAHI)")
    
    return train_files, val_files, test_files

# Eseguiamo setup e split con le percentuali richieste (70/20/10)
setup_directories()
TRAIN_FILES, VAL_FILES, TEST_FILES = split_dataset(val_percent=0.20, test_percent=0.10)

In [ ]:
""" FASE 3: Funzioni Helper per Conversione
Questa è la matematica. Serve una funzione che prenda i box in pixel e li trasformi nel formato 0.5 0.5 0.1 0.2 richiesto da YOLO. """

def to_yolo_format(box, img_w, img_h):
    """Converte [xmin, ymin, xmax, ymax] in [x_center, y_center, w, h] normalizzati"""
    xmin, ymin, xmax, ymax = box
    
    # Calcolo centro e dimensioni
    dw = 1. / img_w
    dh = 1. / img_h
    
    w = xmax - xmin
    h = ymax - ymin
    x_center = xmin + (w / 2.0)
    y_center = ymin + (h / 2.0)
    
    # Normalizzazione (0-1)
    x_center *= dw
    y_center *= dh
    w *= dw
    h *= dh
    
    return x_center, y_center, w, h

def save_chip_data(chip_img, boxes, classes, split_type, base_filename, chip_idx):
    """Salva l'immagine jpg e il file txt corrispondente"""
    
    # Nome base univoco per questa chip
    filename = f"{base_filename.replace('.tif', '')}_chip_{chip_idx}"
    
    # Percorsi di salvataggio
    img_path = f"{OUTPUT_DIR}/{split_type}/images/{filename}.jpg"
    txt_path = f"{OUTPUT_DIR}/{split_type}/labels/{filename}.txt"
    
    has_objects = False
    valid_lines = []

    # Processa ogni box nella chip
    if boxes is not None:
        # Controllo se è un array di zeri (nessun box)
        if not (len(boxes) == 1 and (boxes[0] == 0).all()):
            for i in range(len(boxes)):
                cls_orig = int(classes[i])
                
                # Se la classe è nella nostra mappa, procedi
                if cls_orig in ID_MAP:
                    new_cls = ID_MAP[cls_orig]
                    xc, yc, w, h = to_yolo_format(boxes[i], CHIP_SIZE, CHIP_SIZE)
                    
                    # Formatta la riga per il file txt
                    line = f"{new_cls} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}"
                    valid_lines.append(line)
                    has_objects = True

    # SALVATAGGIO:
    # Salviamo se ha oggetti OPPURE se vogliamo salvare sfondi vuoti (qui salvo solo se ha oggetti per semplicità)
    if has_objects:
        # 1. Salva Immagine
        img = Image.fromarray(chip_img)
        img.save(img_path)
        
        # 2. Salva Label
        with open(txt_path, 'w') as f:
            f.write('\n'.join(valid_lines))

In [ ]:
""" FASE 4: Il Motore (Main Loop)
Questo è il blocco principale che impiegherà del tempo a girare. Carica l'immagine grande, la chippa e chiama la funzione di salvataggio. 
------------------NUOVA AGGIUNTA: IL TEST SET NON LO CROPPO--------------------------
"""

# 1. Carica tutte le etichette in memoria una volta sola (ci mette un po')
print("Caricamento GeoJSON globale...")
coords, chips, classes = wv.get_labels(GEOJSON_PATH)

def process_files(file_list, split_type):
    """Itera sulla lista di file, chippa e salva"""
    print(f"Inizio elaborazione set: {split_type.upper()}...")
    
    for fname in tqdm(file_list):
        try:
            # A. Carica Immagine Originale
            img_path = os.path.join(IMAGES_DIR, fname)
            arr = wv.get_image(img_path)
            
            # B. Filtra le label per QUESTA immagine specifica
            # Usa la maschera booleana numpy (veloce)
            idx = (chips == fname)
            im_coords = coords[idx]
            im_classes = classes[idx]
            
            # C. Crea le Chips (640x640)
            c_imgs, c_boxes, c_classes = wv.chip_image(arr, im_coords, im_classes, shape=(CHIP_SIZE, CHIP_SIZE))
            
            # D. Salva ogni chip
            for k in range(len(c_imgs)):
                save_chip_data(
                    chip_img=c_imgs[k],
                    boxes=c_boxes.get(k),
                    classes=c_classes.get(k),
                    split_type=split_type,
                    base_filename=fname,
                    chip_idx=k
                )
                
        except Exception as e:
            print(f"Errore su {fname}: {e}")

# --- AVVIO PROCESSO ---
# Elabora Training
process_files(TRAIN_FILES, 'train')

# Elabora Validation
process_files(VAL_FILES, 'val')

In [ ]:
""" FASE 5: Creazione YAML
Infine, generiamo il file che passerai a YOLO per iniziare l'addestramento. """

import yaml

def create_yaml_file():
    yaml_content = {
        'path': os.path.abspath(OUTPUT_DIR), # Usa path assoluto per sicurezza
        'train': 'train/images',
        'val': 'val/images',
        'nc': len(CLASS_NAMES),
        'names': CLASS_NAMES
    }
    
    yaml_path = f"{OUTPUT_DIR}/custom_xview.yaml"
    
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)
        
    print(f"File YAML creato con successo: {yaml_path}")
    print("Controlla che 'names' inizi con:", CLASS_NAMES[:3])

create_yaml_file()

# TRAIN

In [ ]:
%%capture
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import os

# --- CONFIGURAZIONE ---
yaml_path = os.path.abspath('datasets/xview_yolo/custom_xview.yaml')

# Parametri di Struttura
MODEL_SIZE = 'yolo11m.pt'
EPOCHS = 150       
BATCH_SIZE = 16   
IMG_SIZE = 640    

def start_training():
    print(f"Caricamento modello {MODEL_SIZE}...")
    model = YOLO(MODEL_SIZE)

    print(f"Avvio training con iperparametri ottimizzati su: {yaml_path}")
    
    # 2. Avvia il training con i parametri da best_hyperparameters.yaml
    results = model.train(
        # Parametri Dataset e Ambiente
        data=yaml_path,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        project='runs/train',
        name='xview_finetune_v1',
        patience=10,
        exist_ok=True,
        verbose=True,

        # Iperparametri di Ottimizzazione (dal tuo YAML)
        #optimizer='AdamW',
        lr0=0.00606,
        lrf=0.00933,
        momentum=0.91256,
        weight_decay=0.00041,
        warmup_epochs=4.35797,
        warmup_momentum=0.79154,
        box=2.49955,
        cls=0.48018,
        dfl=1.59442,

        # Iperparametri di Augmentation (dal tuo YAML)
        hsv_h=0.0184,
        hsv_s=0.62745,
        hsv_v=0.30035,
        degrees=0.0,
        translate=0.09503,
        scale=0.54519,
        shear=0.0,
        perspective=0.0,
        flipud=0.0,
        fliplr=0.49012,
        bgr=0.0,
        mosaic=1.0,
        mixup=0.0,
        cutmix=0.0,
        copy_paste=0.0,
        close_mosaic=7 # Come indicato nello YAML
    )
    
    print("Training completato!")
    print(f"I risultati e i pesi migliori sono salvati in: runs/train/xview_finetune_v1/weights/best.pt")

# --- ESEGUI ---
if __name__ == '__main__':
    start_training()

In [ ]:
""" Se la curva della Loss scende e quella della mAP sale, stai andando alla grande. """

from IPython.display import Image, display
import os

# Assicurati che il percorso corrisponda al 'name' dato nel training
# Esempio: se hai usato name='xview_finetune_v1', il percorso sarà:
results_path = 'runs/train/xview_finetune_v1/results.png'

if os.path.exists(results_path):
    print("Visualizzazione grafici di training:")
    display(Image(filename=results_path))
else:
    print(f"File non trovato in: {results_path}. Controlla se il training è finito.")

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from ultralytics import YOLO

def predict_and_compare(model_path, val_images_dir, val_labels_dir, class_names=None):
    """
    Pesca una chip casuale dal validation set, fa una predizione e confronta
    il risultato con le etichette originali.
    """
    # 1. Carica il modello addestrato
    model = YOLO(model_path)
    
    # 2. Scegli una immagine a caso dalla cartella val
    img_files = [f for f in os.listdir(val_images_dir) if f.endswith('.jpg')]
    if not img_files:
        print("Nessuna immagine trovata in val!")
        return
        
    random_file = random.choice(img_files)
    img_path = os.path.join(val_images_dir, random_file)
    label_path = os.path.join(val_labels_dir, random_file.replace('.jpg', '.txt'))
    
    # 3. Esegui la PREDIZIONE (Inference)
    # conf=0.25 significa che mostra solo box con sicurezza > 25%
    results = model.predict(img_path, conf=0.25, verbose=False)[0]

    # --- PLOTTING ---
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    
    # Immagine Originale per entrambi i plot
    img = Image.open(img_path)
    w_img, h_img = img.size
    
    # A. PLOT SINISTRO: Ground Truth (Verità)
    ax[0].imshow(img)
    ax[0].set_title(f"Ground Truth (Realtà)\n{random_file}")
    ax[0].axis('off')
    
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                cls, nx, ny, nw, nh = map(float, line.split())
                # Denormalizza coordinate (YOLO 0-1 -> Pixel)
                w, h = nw * w_img, nh * h_img
                x, y = (nx * w_img) - w/2, (ny * h_img) - h/2
                
                # Disegna Box Verde (Realtà)
                rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='lime', facecolor='none')
                ax[0].add_patch(rect)
                
                # Testo
                txt = class_names[int(cls)] if class_names else str(int(cls))
                ax[0].text(x, y-2, txt, color='white', fontsize=9, weight='bold', 
                           bbox=dict(facecolor='lime', alpha=0.5, edgecolor='none'))
    else:
        ax[0].text(10, 10, "Nessun oggetto (Background)", color='white')

    # B. PLOT DESTRO: Predizione Modello
    # model.predict restituisce già un oggetto plot facile da usare, ma qui lo facciamo a mano per controllo
    ax[1].imshow(img)
    ax[1].set_title(f"Predizione YOLOv11\nConfidenza > 25%")
    ax[1].axis('off')
    
    # Estraiamo i box predetti
    for box in results.boxes:
        # Coordinate box (xyxy format è il default di results)
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf)
        cls = int(box.cls)
        
        # Disegna Box Rosso (Predizione)
        w, h = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), w, h, linewidth=2, edgecolor='red', facecolor='none')
        ax[1].add_patch(rect)
        
        # Testo con confidenza
        name = results.names[cls]
        label = f"{name} {conf:.2f}"
        ax[1].text(x1, y1-2, label, color='white', fontsize=9, weight='bold',
                   bbox=dict(facecolor='red', alpha=0.5, edgecolor='none'))

    plt.tight_layout()
    plt.show()

# --- ESEMPIO DI USO ---
# Assicurati di passare il percorso corretto al file best.pt che trovi in runs/train/...
predict_and_compare(
     model_path='runs/train/xview_finetune_v1/weights/best.pt', 
     val_images_dir='datasets/xview_yolo/val/images', 
     val_labels_dir='datasets/xview_yolo/val/labels',
     class_names=CLASS_NAMES # La lista nomi che hai creato prima
 )

# INFERENZA CON SAHI 

In [ ]:
%%capture
!pip install sahi 

In [ ]:
""" -------------NUOVA AGGIUNTA: FARO' INFERENZA SUL TEST SET --------------------- """
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.cv import read_image
from IPython.display import Image
import os, random

# 1. Carica il tuo modello YOLO11 personalizzato
detection_model = AutoDetectionModel.from_pretrained(
    model_type="ultralytics",
    model_path="runs/train/xview_finetune_v1/weights/best.pt",  # any yolov8/yolov9/yolo11/yolo12/rt-detr det model is supported
    confidence_threshold=0.25,
    device="cuda:0",  # or 'cuda:0' if GPU is available
)

# VERIFICA PRELIMINARE
# Assicuriamoci che la lista TEST_FILES esista in memoria dalla cella precedente
if 'TEST_FILES' not in locals() or not TEST_FILES:
    raise ValueError("Errore: La lista TEST_FILES non è definita! Esegui la cella di 'Split Dataset' prima di questa.")

# 2. Carica l'immagine grande di xView
image_folder = "/kaggle/input/xview-dataset/train_images/train_images"
supported_extensions = ('.tif', '.tiff', '.jpg', '.jpeg', '.png')

"""all_images = [f for f in os.listdir(image_folder) if f.lower().endswith(supported_extensions)]
if not all_images:
    raise FileNotFoundError(f"Nessuna immagine trovata in {image_folder}")

random_image_name = random.choice(all_images)
random_image_path = os.path.join(image_folder, random_image_name)"""

# Qui sta la magia: invece di os.listdir, usiamo la lista filtrata
random_image_name = random.choice(TEST_FILES)
random_image_path = os.path.join(image_folder, random_image_name)

print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(all_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

# CALCOLO DELLE METRICHE IN INFERENZA

In [ ]:
""" FASE 6: Calcolo Metriche su Test Set (mAP, Precision, Recall) con SAHI
Questo script converte Ground Truth e Predizioni in formato COCO e calcola le metriche.
"""
import json
import time
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
import numpy as np
import PIL

# --- 1. CONFIGURAZIONE ---
# Assicuriamoci di avere le variabili globali caricate (coords, chips, classes, ID_MAP, TEST_FILES)
if 'TEST_FILES' not in locals() or 'coords' not in locals():
    raise ValueError("Mancano le variabili del dataset. Esegui le celle precedenti!")

def create_coco_gt(test_files, image_folder):
    """Crea il dizionario COCO per il Ground Truth delle sole immagini di Test (Corretto con 'info')"""
    
    # --- CORREZIONE QUI: Aggiunta chiave 'info' e 'licenses' richieste da pycocotools ---
    coco_gt = {
        "info": {
            "description": "xView Custom Test Set",
            "url": "",
            "version": "1.0",
            "year": 2024,
            "contributor": "",
            "date_created": ""
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }
    # -------------------------------------------------------------------------------------
    
    # Crea Categorie
    unique_yolo_ids = sorted(list(set(ID_MAP.values())))
    for y_id in unique_yolo_ids:
        coco_gt["categories"].append({"id": int(y_id), "name": str(y_id), "supercategory": "object"})

    ann_id_counter = 1
    
    print("Generazione Ground Truth COCO per il Test Set (con metadati corretti)...")
    
    for file_idx, fname in enumerate(tqdm(test_files)):
        image_id = file_idx  
        
        # Leggiamo dimensioni reali
        img_full_path = os.path.join(image_folder, fname)
        try:
            with Image.open(img_full_path) as img:
                width, height = img.size
        except Exception as e:
            # print(f"Errore lettura {fname}: {e}. Uso default.") # Commentato per pulizia output
            width, height = 3000, 3000 

        coco_gt["images"].append({
            "id": image_id,
            "file_name": fname,
            "width": width,
            "height": height
        })
        
        # Aggiungi Annotazioni
        idx = (chips == fname)
        im_coords = coords[idx]
        im_classes = classes[idx]
        
        for box, cls_orig in zip(im_coords, im_classes):
            if int(cls_orig) in ID_MAP:
                cls_yolo = ID_MAP[int(cls_orig)]
                
                xmin, ymin, xmax, ymax = box
                w = xmax - xmin
                h = ymax - ymin
                
                coco_gt["annotations"].append({
                    "id": ann_id_counter,
                    "image_id": image_id,
                    "category_id": int(cls_yolo),
                    "bbox": [float(xmin), float(ymin), float(w), float(h)],
                    "area": float(w * h),
                    "iscrowd": 0
                })
                ann_id_counter += 1
                
    return coco_gt
   

# --- 2. ESECUZIONE INFERENZA SU TUTTO IL TEST SET ---
def run_sahi_inference_and_export(test_files, image_folder, model):
    """Esegue SAHI su tutte le immagini di test e raccoglie i risultati in formato COCO"""
    coco_results = []
    
    print("\nAvvio Inferenza SAHI su Test Set...")
    for file_idx, fname in enumerate(tqdm(test_files)):
        image_id = file_idx # Deve corrispondere all'ID nel GT
        img_path = os.path.join(image_folder, fname)
        
        # Inferenza SAHI
        result = get_sliced_prediction(
            img_path,
            model,
            slice_height=640,
            slice_width=640,
            overlap_height_ratio=0.2,
            overlap_width_ratio=0.2,
            verbose=0 # Silenzioso
        )
        
        # Estrai predizioni
        for pred in result.object_prediction_list:
            # bbox in SAHI è [xmin, ymin, xmax, ymax]
            x_min, y_min, x_max, y_max = pred.bbox.to_xyxy()
            w = x_max - x_min
            h = y_max - y_min
            
            score = pred.score.value
            category_id = pred.category.id # Questo è l'ID YOLO (0..N) restituito dal modello
            
            coco_results.append({
                "image_id": image_id,
                "category_id": int(category_id),
                "bbox": [float(x_min), float(y_min), float(w), float(h)],
                "score": float(score)
            })
            
    return coco_results

# --- MAIN FLOW ---

# 1. Prepara i dati GT
gt_json = create_coco_gt(TEST_FILES, "/kaggle/input/xview-dataset/train_images/train_images")

# Salva GT su disco (necessario per COCO API)
with open('test_gt.json', 'w') as f:
    json.dump(gt_json, f)

# 2. Ottieni Predizioni (Usa il 'detection_model' che hai caricato nella cella precedente)
# Attenzione: Questo loop può impiegare TEMPO a seconda di quante immagini di test hai.
# Per test veloce, puoi ridurre TEST_FILES[:10]
predictions = run_sahi_inference_and_export(TEST_FILES, "/kaggle/input/xview-dataset/train_images/train_images", detection_model)

# Salva Predizioni su disco
with open('test_pred.json', 'w') as f:
    json.dump(predictions, f)

# --- 3. CALCOLO METRICHE ---
print("\n--- RISULTATI VALUTAZIONE ---")

# Inizializza COCO
cocoGt = COCO('test_gt.json')
cocoDt = cocoGt.loadRes('test_pred.json')

# Esegui COCOeval
cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

print("\nLegenda:")
print("AP @ .50:95 = mAP (Metrica primaria)")
print("AP @ .50    = mAP50 (Spesso usata in YOLO)")
print("AR @ 100    = Recall (per un massimo di 100 detection per immagine)")

# Per ricordare gli split

In [ ]:
print(TEST_FILES)

In [ ]:
print(VAL_FILES)

In [ ]:
import time
i=0
print('Notebook finito, itero nella tua attesa master')
while(True):
    print('Numero ore attese: ',i)
    i=i+1
    time.sleep(3600)

# Altro tentativo di valutazione

In [ ]:
""" QUESTA è UNA VALUTAZIONE SEMPRE SUL DATASET TAGLIATO """
from ultralytics import YOLO

# Load the model
model = YOLO("/kaggle/working/runs/train/xview_finetune_v1/weights/best.pt")

# Run the evaluation
results = model.val(data="/kaggle/working/datasets/xview_yolo/custom_xview.yaml")

# Print specific metrics
print("Class indices with average precision:", results.ap_class_index)
print("Average precision for all classes:", results.box.all_ap)
print("Average precision:", results.box.ap)
print("Average precision at IoU=0.50:", results.box.ap50)
print("Class indices for average precision:", results.box.ap_class_index)
print("Class-specific results:", results.box.class_result)
print("F1 score:", results.box.f1)
print("F1 score curve:", results.box.f1_curve)
print("Overall fitness score:", results.box.fitness)
print("Mean average precision:", results.box.map)
print("Mean average precision at IoU=0.50:", results.box.map50)
print("Mean average precision at IoU=0.75:", results.box.map75)
print("Mean average precision for different IoU thresholds:", results.box.maps)
print("Mean results for different metrics:", results.box.mean_results)
print("Mean precision:", results.box.mp)
print("Mean recall:", results.box.mr)
print("Precision:", results.box.p)
print("Precision curve:", results.box.p_curve)
print("Precision values:", results.box.prec_values)
print("Specific precision metrics:", results.box.px)
print("Recall:", results.box.r)
print("Recall curve:", results.box.r_curve)

In [ ]:
""" REPORT PIù LEGGIBILE """
import pandas as pd

# 1. Metriche Globali
print(f"{'METRICA':<20} | {'VALORE':<10}")
print("-" * 35)
print(f"{'mAP @ 50-95':<20} | {results.box.map:.4f}")
print(f"{'mAP @ 50':<20} | {results.box.map50:.4f}")
print(f"{'Precision (Media)':<20} | {results.box.mp:.4f}")
print(f"{'Recall (Media)':<20} | {results.box.mr:.4f}")
print("-" * 35)

# 2. Analisi per Classe (Top & Flop)
# Creiamo un DataFrame per leggere meglio i dati
# Nota: Assicurati che 'model.names' contenga i nomi delle classi
class_names = results.names  # O la tua lista CLASS_NAMES
maps_per_class = results.box.maps  # mAP 50-95 per ogni classe

df_classes = pd.DataFrame({
    'Classe': class_names.values(), # o solo class_names se è una lista
    'mAP 50-95': maps_per_class
})

# Ordiniamo per vedere le migliori e le peggiori
df_sorted = df_classes.sort_values(by='mAP 50-95', ascending=False)

print("\n--- MIGLIORI 5 CLASSI ---")
print(df_sorted.head(5).to_string(index=False))

print("\n--- PEGGIORI 5 CLASSI ---")
print(df_sorted.tail(5).to_string(index=False))

In [ ]:
# Print specific metrics
print("Class indices with average precision:", results.ap_class_index)
print("Average precision for all classes:", results.box.all_ap)
print("Average precision:", results.box.ap)
print("Average precision at IoU=0.50:", results.box.ap50)
print("Class indices for average precision:", results.box.ap_class_index)
print("Class-specific results:", results.box.class_result)
print("F1 score:", results.box.f1)
print("F1 score curve:", results.box.f1_curve)
print("Overall fitness score:", results.box.fitness)
print("Mean average precision:", results.box.map)
print("Mean average precision at IoU=0.50:", results.box.map50)
print("Mean average precision at IoU=0.75:", results.box.map75)
print("Mean average precision for different IoU thresholds:", results.box.maps)
print("Mean results for different metrics:", results.box.mean_results)
print("Mean precision:", results.box.mp)
print("Mean recall:", results.box.mr)
print("Precision:", results.box.p)
print("Precision curve:", results.box.p_curve)
print("Precision values:", results.box.prec_values)
print("Specific precision metrics:", results.box.px)
print("Recall:", results.box.r)
print("Recall curve:", results.box.r_curve)

# Tentativo al fly di validazione sul test set grosso


In [ ]:
""" FASE 6: Calcolo Metriche Ufficiali su TEST SET (Immagini Intere)
Non serve YAML: usiamo direttamente i percorsi dei file e la lista TEST_FILES.
"""
import json
import os
import time
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
from PIL import Image
from sahi.predict import get_sliced_prediction

# --- 1. CONFIGURAZIONE PERCORSI ---
# Percorso delle immagini originali (preso dal tuo screenshot)
# NOTA: In xView su Kaggle spesso il path è doppiato 'train_images/train_images'
ORIGINAL_IMAGES_DIR = "/kaggle/input/xview-dataset/train_images/train_images"

# Verifica che la cartella esista
if not os.path.exists(ORIGINAL_IMAGES_DIR):
    raise FileNotFoundError(f"Non trovo la cartella: {ORIGINAL_IMAGES_DIR}")

# Verifica variabili necessarie
if 'TEST_FILES' not in locals() or 'coords' not in locals():
    raise ValueError("ERRORE: Mancano le variabili 'TEST_FILES' o 'coords'. Esegui le celle di Split e Caricamento dati!")

print(f"Userò {len(TEST_FILES)} immagini per il test (dal 10% di split creato prima).")

# --- 2. FUNZIONE GENERAZIONE GROUND TRUTH (COCO) ---
def create_coco_gt_from_memory(test_files, image_folder):
    """
    Crea il JSON del Ground Truth convertendo al volo le annotazioni
    presenti nelle variabili globali 'chips', 'coords', 'classes'.
    """
    coco_gt = {
        "info": {"description": "xView Test Set"}, # Necessario per pycocotools
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # Crea Categorie
    unique_ids = sorted(list(set(ID_MAP.values())))
    for y_id in unique_ids:
        # Usa il nome della classe se disponibile, altrimenti l'ID
        c_name = CLASS_NAMES[y_id] if 'CLASS_NAMES' in locals() else str(y_id)
        coco_gt["categories"].append({"id": int(y_id), "name": c_name, "supercategory": "object"})

    ann_id = 1
    
    print("Generazione file Ground Truth (GT)...")
    for idx_img, fname in enumerate(tqdm(test_files)):
        full_path = os.path.join(image_folder, fname)
        
        # 1. Recupera dimensioni reali immagine (importante per mAP corretta)
        try:
            with Image.open(full_path) as img:
                width, height = img.size
        except:
            width, height = 3000, 3000 # Fallback
            
        coco_gt["images"].append({
            "id": idx_img, 
            "file_name": fname, 
            "width": width, 
            "height": height
        })
        
        # 2. Recupera le label dalle variabili globali di xView
        # Trova gli indici dove il nome file corrisponde
        mask = (chips == fname)
        im_boxes = coords[mask]   # [xmin, ymin, xmax, ymax]
        im_classes = classes[mask]
        
        for box, cls_orig in zip(im_boxes, im_classes):
            if int(cls_orig) in ID_MAP:
                cls_yolo = ID_MAP[int(cls_orig)]
                xmin, ymin, xmax, ymax = box
                w = xmax - xmin
                h = ymax - ymin
                
                coco_gt["annotations"].append({
                    "id": ann_id,
                    "image_id": idx_img,
                    "category_id": int(cls_yolo),
                    "bbox": [float(xmin), float(ymin), float(w), float(h)],
                    "area": float(w*h),
                    "iscrowd": 0
                })
                ann_id += 1
                
    return coco_gt

# --- 3. FUNZIONE INFERENZA (SAHI) ---
def run_sahi_evaluation(test_files, image_folder, model):
    coco_preds = []
    
    print("\nAvvio Inferenza SAHI...")
    for idx_img, fname in enumerate(tqdm(test_files)):
        full_path = os.path.join(image_folder, fname)
        
        # Inferenza SAHI
        # slice_height/width: usa 640 (come il training)
        # overlap: 0.2 standard, puoi provare 0.5 per migliorare i risultati
        result = get_sliced_prediction(
            full_path,
            model,
            slice_height=640,
            slice_width=640,
            overlap_height_ratio=0.2,
            overlap_width_ratio=0.2,
            verbose=0,
            postprocess_match_metric="IOS" # Gestione box sovrapposti nel merge
        )
        
        # Estrai predizioni
        for pred in result.object_prediction_list:
            x, y, x2, y2 = pred.bbox.to_xyxy()
            w = x2 - x
            h = y2 - y
            
            coco_preds.append({
                "image_id": idx_img,
                "category_id": int(pred.category.id),
                "bbox": [float(x), float(y), float(w), float(h)],
                "score": float(pred.score.value)
            })
            
    return coco_preds

# ==========================================
#         ESECUZIONE PRINCIPALE
# ==========================================

# A. Genera e Salva Ground Truth
gt_data = create_coco_gt_from_memory(TEST_FILES, ORIGINAL_IMAGES_DIR)
with open("test_gt.json", "w") as f:
    json.dump(gt_data, f)

# B. Genera e Salva Predizioni (SAHI)
# Assicurati che 'detection_model' sia caricato (cella precedente)
pred_data = run_sahi_evaluation(TEST_FILES, ORIGINAL_IMAGES_DIR, detection_model)
with open("test_pred.json", "w") as f:
    json.dump(pred_data, f)

# C. Calcola Metriche
print("\n--- CALCOLO METRICHE ---")
cocoGt = COCO("test_gt.json")
cocoDt = cocoGt.loadRes("test_pred.json")

cocoEval = COCOeval(cocoGt, cocoDt, "bbox")
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()